# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
## Clean data and label encode 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [6]:
## Encode string labels into numbers so model can read and make use of the columns 
le.fit(df["home_ownership"])
df["home_ownership"] = le.transform(df["home_ownership"])

le.fit(df["verification_status"])
df["verification_status"] = le.transform(df["verification_status"])

le.fit(df["pymnt_plan"])
df["pymnt_plan"] = le.transform(df["pymnt_plan"])

le.fit(df["initial_list_status"])
df["initial_list_status"] = le.transform(df["initial_list_status"])

le.fit(df["next_pymnt_d"])
df["next_pymnt_d"] = le.transform(df["next_pymnt_d"])

le.fit(df["application_type"])
df["application_type"] = le.transform(df["application_type"])

le.fit(df["hardship_flag"])
df["hardship_flag"] = le.transform(df["hardship_flag"])

le.fit(df["debt_settlement_flag"])
df["debt_settlement_flag"] = le.transform(df["debt_settlement_flag"])

## Split the Data into Training and Testing

In [7]:
# Create our features
x_cols = [i for i in df.columns if i not in ('loan_status','issue_d',)]
X = df[x_cols]

# Create our target
y = df['loan_status']

In [8]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.0,21.778153,0.217766,0.497697,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.0,20.199244,0.718367,0.758122,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.0,0.000000,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.0,13.890000,0.000000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,0.0,19.760000,0.000000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,0.0,26.660000,0.000000,1.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,0.0,999.000000,18.000000,5.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [9]:
# Check the balance of our target values
# YOUR CODE HERE
Counter(y)

Counter({'low_risk': 68470, 'high_risk': 347})

In [10]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [11]:
# Create the StandardScaler instance
# YOUR CODE HERE
scaler = StandardScaler()

In [12]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

In [13]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [14]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)

In [15]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.995640802092415


In [16]:
# Display the confusion matrix
# YOUR CODE HERE
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,32,69
Actual 1,6,17098


In [17]:
## Print balanced classification report (not required)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

   high_risk       0.84      0.32      0.46       101
    low_risk       1.00      1.00      1.00     17104

    accuracy                           1.00     17205
   macro avg       0.92      0.66      0.73     17205
weighted avg       1.00      1.00      0.99     17205



In [18]:
# Print the imbalanced classification report
# YOUR CODE HERE
cr_imb = classification_report_imbalanced(y_test, predictions,output_dict=True)
recall_RF = cr_imb['avg_rec']
geo_RF = cr_imb['avg_geo']
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.32      1.00      0.46      0.56      0.30       101
   low_risk       1.00      1.00      0.32      1.00      0.56      0.34     17104

avg / total       1.00      1.00      0.32      0.99      0.56      0.34     17205



In [19]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
importances = rf_model.feature_importances_
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted

[(0.06798196265776532, 'total_pymnt_inv'),
 (0.06508522964456012, 'last_pymnt_amnt'),
 (0.062213223605891835, 'total_rec_int'),
 (0.06133188103056622, 'total_rec_prncp'),
 (0.05635927692678298, 'total_pymnt'),
 (0.019552389522142076, 'dti'),
 (0.018653265009093273, 'max_bal_bc'),
 (0.018006415861758076, 'mo_sin_old_rev_tl_op'),
 (0.016727643926202424, 'out_prncp'),
 (0.01668355726005984, 'installment'),
 (0.016506458137203094, 'out_prncp_inv'),
 (0.016173289373868558, 'total_bal_ex_mort'),
 (0.01615360501883616, 'mo_sin_old_il_acct'),
 (0.016140718611097858, 'tot_hi_cred_lim'),
 (0.016067116010189394, 'revol_bal'),
 (0.015886792866437113, 'bc_open_to_buy'),
 (0.015542727018787584, 'bc_util'),
 (0.015204871604237506, 'avg_cur_bal'),
 (0.015160200417539542, 'total_bal_il'),
 (0.015027646259473382, 'tot_cur_bal'),
 (0.014863703164238634, 'total_bc_limit'),
 (0.014399001785262053, 'annual_inc'),
 (0.01401963510998958, 'total_il_high_credit_limit'),
 (0.013140514606775204, 'mths_since_rcnt_

### Easy Ensemble Classifier

In [20]:
# Train the Classifier
# YOUR CODE HERE
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=500, random_state=1)
eec = eec.fit(X_train_scaled, y_train)
y_pred = eec.predict(X_test_scaled)

In [21]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
acc_score = accuracy_score(y_test, y_pred)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.7792502179598954


In [22]:
# Display the confusion matrix
# YOUR CODE HERE
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,69,32
Actual 1,3766,13338


In [23]:
# Print the imbalanced classification report
# YOUR CODE HERE
cr_imb = classification_report_imbalanced(y_test, y_pred,output_dict=True)
recall_eec = cr_imb['avg_rec']
geo_eec = cr_imb['avg_geo']
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.68      0.78      0.04      0.73      0.53       101
   low_risk       1.00      0.78      0.68      0.88      0.73      0.54     17104

avg / total       0.99      0.78      0.68      0.87      0.73      0.54     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Random Forest Model, by a lot. 

2. Which model had the best recall score?

    Random Forest Model. 

3. Which model had the best geometric mean score?

    The EEC Model. 

4. What are the top three features?

    total_pymnt_inv, and
    last_pymnt_amnt, and 
    total_rec_int

In [24]:
recall_RF

0.995640802092415

In [25]:
recall_eec

0.7792502179598954

In [26]:
geo_RF

0.5627792996859822

In [27]:
geo_eec

0.7298949705437634